In [1]:
import cv2
import json
from datetime import timedelta

In [18]:
def template_matching_video(video_path, logo_path, threshold=0.5):
    # Carregar o vídeo e a logotipo alvo
    print(video_path)
    print(threshold)
    video = cv2.VideoCapture(video_path)
    template = cv2.imread(logo_path, cv2.IMREAD_COLOR)

    # Obter as dimensões da logotipo
    w, h = template.shape[1], template.shape[0]

    # Informações do vídeo
    fps = video.get(cv2.CAP_PROP_FPS)  # Frames por segundo
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))  # Número total de frames
    duration = frame_count / fps  # Duração total do vídeo em segundos

    detections = []
    detecting = False
    start_time = None

    frame_idx = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Aplicar o template matching no frame atual
        result = cv2.matchTemplate(frame, template, cv2.TM_CCORR_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

        # Verificar se a similaridade atingiu o threshold
        if max_val >= threshold:
            if not detecting:
                # Início da detecção
                detecting = True
                start_time = frame_idx / fps
        else:
            if detecting:
                # Fim da detecção
                detecting = False
                end_time = frame_idx / fps
                detections.append(
                    {
                        "start": str(timedelta(seconds=start_time)),
                        "end": str(timedelta(seconds=end_time)),
                    }
                )

        frame_idx += 1

    # Verificar se a detecção estava ativa até o final do vídeo
    if detecting:
        end_time = frame_count / fps
        detections.append(
            {
                "start": str(timedelta(seconds=start_time)),
                "end": str(timedelta(seconds=end_time)),
            }
        )

    video.release()

    # Salvar os resultados em um arquivo JSON
    # with open(output_json_path, "w") as json_file:
    #     json.dump(detections, json_file, indent=4)

    return print(detections)

In [20]:
video_path = "../videos/15s-canto-esquerdo-pequeno.mp4"
logo_path = "../target/logo_target.png"
template_matching_video(video_path, logo_path)

../videos/15s-canto-esquerdo-pequeno.mp4
0.5
[{'start': '0:00:00', 'end': '0:00:15.033333'}]
